# VCintModel

Machine information 

In [1]:
versioninfo()

Julia Version 1.2.0
Commit c6da87ff4b (2019-08-20 00:03 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin18.6.0)
  CPU: Intel(R) Core(TM) i5-6267U CPU @ 2.90GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-6.0.1 (ORCJIT, skylake)


## Data

Suppose we have the following variables:

* `y`: $324$-by-$1$ vector of phenotype of interest 
* `X`: $324$-by-$1$ vector indicating sex of each individual 
* `G`: $150$-by-$1$ vector whose $i$-th element is a $324$-by-$q_i$ genotype matrix (matrix of minor allele counts) for gene $i,$ where $i=1,\ldots, 150$ and $q_i$ is the number of variants for gene $i$
* `trtvec`: $324$-by-$1$ vector indicating treatment status of each individual. 

These variables are saved in `.jld2` file. First, read in variables using `JLD2` package. 

In [4]:
using JLD2
@load "/Users/juhyun-kim/Box Sync/workspace/vcselect/codebase/julia/VarianceComponentSelect.jl/docs/SNPset_interaction.jld2" y y2 X G V trtvec

6-element Array{Symbol,1}:
 :y     
 :y2    
 :X     
 :G     
 :V     
 :trtvec

Our phenotype vector: 

In [5]:
y

324-element Array{Float64,1}:
 -0.40911273847444835 
 -0.0597995644528963  
  0.02798994878695149 
 -0.5810287222548617  
  0.4325440331155418  
  0.048256153207898195
 -0.27027696763685594 
  0.10490944338119751 
 -0.7799009563553139  
 -0.20593444247247622 
  0.5520065403740653  
 -0.10092686337307819 
  0.5983104367265357  
  ⋮                   
  0.03739468023738035 
  0.47352506877371714 
 -0.22836115042409924 
 -0.3876214190551975  
  0.10326243849550518 
 -0.4882830145406528  
 -0.06903684417417116 
  0.09798041795966334 
 -0.1480073641765541  
 -0.29900800903355274 
 -0.008878790322644575
 -0.09571535379121696 

Covariate matrix indicating sex:

In [6]:
X

324-element Array{Float64,1}:
 1.0
 2.0
 2.0
 1.0
 2.0
 1.0
 2.0
 2.0
 2.0
 1.0
 2.0
 1.0
 1.0
 ⋮  
 1.0
 1.0
 2.0
 2.0
 2.0
 1.0
 1.0
 2.0
 1.0
 1.0
 1.0
 2.0

Vector of genotype matrix for each gene: 

In [7]:
G

150-element Array{Array{Float64,2},1}:
 [2.0 0.0 … 2.0 2.0; 2.0 1.0 … 1.4485981308411215 2.0; … ; 2.0 0.0 … 1.0 2.0; 2.0 0.0 … 2.0 2.0]
 [2.0 2.0 … 2.0 2.0; 2.0 1.0 … 1.0 0.0; … ; 2.0 1.0 … 1.0 0.0; 2.0 2.0 … 1.0 2.0]               
 [2.0 2.0 … 0.0 2.0; 2.0 2.0 … 2.0 2.0; … ; 2.0 2.0 … 1.0 2.0; 2.0 2.0 … 0.0 1.0]               
 [2.0 1.0 … 0.0 2.0; 2.0 1.0 … 0.0 2.0; … ; 2.0 0.0 … 1.0 2.0; 1.0 1.0 … 1.0 2.0]               
 [1.0 2.0 … 0.0 2.0; 1.0 2.0 … 1.0 2.0; … ; 0.0 2.0 … 1.0 2.0; 2.0 2.0 … 0.0 2.0]               
 [2.0 2.0 … 1.0 2.0; 2.0 2.0 … 1.0 2.0; … ; 2.0 2.0 … 1.0 2.0; 2.0 2.0 … 1.0 2.0]               
 [2.0 1.0 … 1.0 2.0; 2.0 1.0 … 1.0 2.0; … ; 2.0 2.0 … 0.0 2.0; 2.0 2.0 … 1.0 2.0]               
 [1.0 1.0 … 1.0 1.0; 1.0 2.0 … 1.0 0.0; … ; 2.0 2.0 … 2.0 1.0; 2.0 2.0 … 2.0 1.0]               
 [2.0 2.0 … 2.0 0.0; 2.0 1.0 … 1.0 0.0; … ; 1.0 2.0 … 2.0 0.0; 2.0 2.0 … 2.0 1.0]               
 [2.0 2.0 … 2.0 1.0; 2.0 2.0 … 2.0 1.0; … ; 2.0 2.0 … 2.0 1.0; 2.0 0.0 … 2.0 1.0]       

Elements in `G` are matrices of size $324\times q_i.$ Number of rows must be the same (n=324) because each row is for each individual. On the other hand, number of columns vary because number of SNPs/variants vary from gene to gene. Here we list different sizes elements of `G` have: 

In [10]:
unique(size.(G))

8-element Array{Tuple{Int64,Int64},1}:
 (324, 44)
 (324, 27)
 (324, 47)
 (324, 14)
 (324, 42)
 (324, 29)
 (324, 30)
 (324, 18)

Vector of treatment status:

In [8]:
trtvec

324-element Array{Int64,1}:
 0
 0
 0
 0
 1
 1
 0
 1
 0
 1
 1
 1
 1
 ⋮
 1
 1
 0
 0
 0
 0
 1
 0
 0
 0
 0
 1

Let us create a diagonal matrix whose elements are from `trtvec`:


In [13]:
using LinearAlgebra
T = Diagonal(trtvec)

324×324 Diagonal{Int64,Array{Int64,1}}:
 0  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  …  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  0  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  0  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  0  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  …  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  0  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  0  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  …  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ 

Now based on what we have, we will create two vectors of covariance matrices:

* `V`: 151-by-1 vector of 324-by-324 matrices 
    - `V[i]` $ = G_i G_i^T / ||G_i G_i^T||_F$ where $i=1,\ldots,150$
    - `V[end]` $= I_{324} / \sqrt{324}$
* `Vint`: 150-by-1 vector of 324-by-324 matrices
    - `Vint[i]` $ = T G_i G_i^T T^T / ||T G_i G_i^T T^T||_F$ where $i = 1,\ldots, 150.$
    
Note that we divide by Frobenius norm ($||\cdot||_F$) to put matrices on the same scale.

In [15]:
n, m = length(y), length(G)
V = Vector{Matrix{Float64}}(undef, m + 1)
Vint = Vector{Matrix{Float64}}(undef, m)
    
for i in 1:m
    V[i] = G[i] * G[i]'
    Vint[i] = T * V[i] * T' 
    V[i] ./= norm(V[i])
    Vint[i] ./= norm(Vint[i])
end 
V[end] = Matrix(I, n, n) ./ √n;

In [19]:
V

151-element Array{Array{Float64,2},1}:
 [0.003291667254666042 0.003021726705196923 … 0.0030853699763006446 0.002995120655146579; 0.003021726705196923 0.0031759677171174796 … 0.0029207450416141385 0.0029624173852930303; … ; 0.0030853699763006446 0.0029207450416141385 … 0.0033536029685539274 0.0030544299750504714; 0.002995120655146579 0.0029624173852930303 … 0.0030544299750504714 0.0034695952143777202]    
 [0.0038736704792971055 0.0031542745331419285 … 0.0034586343565152725 0.00367998695533225; 0.0031542745331419285 0.0030989363834376844 … 0.0032372817576982954 0.0030989363834376844; … ; 0.0034586343565152725 0.0032372817576982954 … 0.0037353251050364945 0.0034032962068110285; 0.00367998695533225 0.0030989363834376844 … 0.0034032962068110285 0.003790663254740739]    
 [0.003134943134737379 0.0029621509934526415 … 0.003011520176676852 0.0027399896689436934; 0.0029621509934526415 0.003283050684410011 … 0.0030608893599010627 0.0028387280353921146; … ; 0.003011520176676852 0.003060889359901

In [20]:
Vint

150-element Array{Array{Float64,2},1}:
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.007022323013197897] 
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.00773729902969992]  
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0064020557433550306]
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.007187137793403356] 
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.007319758091161073] 
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.007135751936863028] 
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.006472228768822936] 
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.007290657333798592] 
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.007352558297122053] 
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0

## Model formulation 

We assume the response vector $y$ is distributed as follows: <a id='modeleqn'></a>

$$y \sim N(X\beta, \sum_{i=1}^{m} \left( \sigma_{i1}^2 V_{i1} +  \sigma_{i2}^2 V_{i2}\right) + \sigma_0^2 I_n ), \hspace{5em} (1)$$

such that each gene/group (indexed by $i$) is associated with two variance components
- $\sigma_{i1}^2$: for the gene/group itself 
- $\sigma_{i2}^2$: for the interaction between gene/group and treatment status indicated by $T$. 

Suppose we want to include/exclude main effects and interaction term together as a pair and identify the pairs that are associated with the response $y.$ In other words, all-in or all-out! This can be achieved by setting up `VCintModel` in `VarianceComponentSelect` package.

## VCintModel

First load the package. 

In [23]:
using VarianceComponentSelect

To perform selection, take 2 steps:

**Step 1 (Construct a model)**. Construct an instance of `VCintModel`, which is the fundamental type for variance component interaction model. It consists of fields 

* `Y`: $n$-by-$1$ responses. 
* `X`: $n$-by-$p$ covariate matrix (if exists).
* `V=(V[1],...,V[m],I)`: a vector of $n$-by-$n$ covariance matrices. The last covariance matrix must be positive definite and usually is the identity matrix.
* `Vint=(Vint[1],...,Vint[m])`: a vector of $n$-by-$n$ covariance matrices. 
* `Σ=(Σ[1],...,Σ[m],Σ[0])`: a vector of initial estimates for variance component parameters. If not supplied, it is set to be a vector of ones by default. In the notation of [(1)](#modeleqn) above, it is equivalent to $(\sigma_{11}^2, \ldots, \sigma_{m1}^2, \sigma_0^2).$
* `Σint=(Σint[1],...,Σint[m])`: a vector of initial estimates for variance component parameters. If not supplied, it is set to be a vector of ones by default. In the notation of [(1)](#modeleqn) above, it is equivalent to $(\sigma_{12}^2, \ldots, \sigma_{m2}^2).$

`VCintModel` can be initialized by 

Let us construct a `VCintModel` using `Y`, `X`, `Vint` and `V`. 

In [24]:
vcm = VCintModel(y, X, V, Vint);

Since we did not provide `Σ` and `Σint`, both have been initialized to be a vector of ones. 

In [25]:
vcm.Σ

151-element Array{Float64,1}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 ⋮  
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

In [26]:
vcm.Σint

150-element Array{Float64,1}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 ⋮  
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

**Step 2 (optimize)**. Call optimization routine `vcselect!`.

Required input argument for executing `vcselect!` is `VCModel`:
    
- `vcm`: `VCModel`.

Keyword Arguments are 

- `penfun`: penalty function. Default is `NoPenalty()`.
- `λ`: tuning parameter. Default is 1.    
- `penwt`: penalty weights. Default is (1,...1,0).
- `standardize`: logical flag for covariance matrix standardization. Default is `true`.
    If true, `V[i]` is standardized by its Frobenius norm, and parameter estimates are 
    returned on the original scale.
- `maxiters`: maximum number of iterations. Default is 1000.
- `tol`: convergence tolerance. Default is `1e-6`.
- `verbose`: display switch. Default is false.
- `checktype`: check argument type switch. Default is true.

<a id='no-penalty'></a>
### No penalty 

First, we call `vcselect!` without specifying any penalty function. By default, it is fit with no penalty (i.e. $\lambda=0$).

In [28]:
vcm_nopen = deepcopy(vcm)
vcm_nopen, obj, niters, = vcselect!(vcm_nopen);

The output of `vcselect!` include

* fitted model 

Estimates can be accessed as below: 

In [29]:
# variance components for main effect 
vcm_nopen.Σ

151-element Array{Float64,1}:
  1.2370956021268487e-103
  1.18459422741841e-14   
  1.4514980803170268e-181
  1.090535287358165e-22  
  6.744718532465077e-61  
 10.565155035553508      
  5.131473929132298e-49  
  2.875236157689849e-48  
  1.941047934124898e-52  
  2.5644485860721365e-14 
  5.690783579703894e-141 
  5.449013992876955e-117 
  2.5104793319889334e-123
  ⋮                      
  2.2447558973276592e-118
  1.4579400632656927e-39 
  6.278029617215636e-75  
  1.0761872636315929e-35 
  2.5959925203022874e-35 
  3.260689817232068e-6   
  4.19984969005698e-101  
  9.337835915840925e-12  
  4.030269129408247      
  8.359367009877688e-118 
 19.489087310922567      
  5.564148840746572e-8   

In [32]:
# variance components for interaction effect 
vcm_nopen.Σint

150-element Array{Float64,1}:
 1.310309666932489e-28  
 3.2419000811015885e-12 
 1.5702290697410488e-55 
 2.672343838280577e-131 
 6.211595722299537e-29  
 3.159125768372407e-98  
 3.722393641006552e-41  
 3.8688656742473736e-100
 1.168083164394658e-25  
 2.21691530955166e-62   
 7.970281510431552e-107 
 8.352279869688793e-116 
 6.3034711280692e-190   
 ⋮                      
 1.2394173201953727e-16 
 4.454553258162834e-75  
 1.9204496423522967e-66 
 6.808362980686468e-47  
 2.0580025119265423e-40 
 3.4591922795688957e-62 
 8.476522782648709e-144 
 9.829626888036323e-70  
 2.2109686410414783e-37 
 4.3962325743466516e-48 
 3.905521953657742e-76  
 3.708715008326754      

In [31]:
# mean regression coefficients 
vcm_nopen.β

1-element Array{Float64,1}:
 -0.053324093482632964

* final objective value

In [33]:
obj

-6.465972564484275

* number of iterations taken to converge 

In [34]:
niters

176

### Selection at specific tuning parameter 

Now we specify penalty function `penfun=L1Penalty()` and regularization parameter value `λ=2.5`. 

In [35]:
vcm_L1 = deepcopy(vcm)
vcm_L1, obj_L1, niters_L1, = vcselect!(vcm_L1; penfun=L1Penalty(), λ=2.0);

In [36]:
vcm_L1.Σ

151-element Array{Float64,1}:
  0.0                 
  0.0                 
  0.0                 
  0.0                 
  0.0                 
  8.349184962746191   
  0.0                 
  0.0                 
  0.0                 
  0.0                 
  0.0                 
  0.0                 
  0.0                 
  ⋮                   
  0.0                 
  0.0                 
  0.0                 
  0.0                 
  0.0                 
  0.0                 
  0.0                 
  0.0                 
  0.0                 
  0.0                 
 17.75523375970695    
  5.564148840746572e-8

In [37]:
vcm_L1.Σint

150-element Array{Float64,1}:
 0.0                   
 0.0                   
 0.0                   
 0.0                   
 0.0                   
 5.919818935419774e-146
 0.0                   
 0.0                   
 0.0                   
 0.0                   
 0.0                   
 0.0                   
 0.0                   
 ⋮                     
 0.0                   
 0.0                   
 0.0                   
 0.0                   
 0.0                   
 0.0                   
 0.0                   
 0.0                   
 0.0                   
 0.0                   
 0.0                   
 2.3867430598010366    

In [38]:
vcm_L1.β

1-element Array{Float64,1}:
 -0.06823101905015315

In [39]:
obj_L1

42.377774228329464

In [40]:
niters_L1

389

## Obtain solution path 

`vcselectpath!` function to compute regularization path for a given penalty at a grid of the regularization parameter lambda values. `vcselectpath!` provides options for users to customize. Keyword arguments for the function are 

- `penfun`: penalty function (e.g. `NoPenalty()`, `L1Penalty()`, `MCPPenalty()`). Default is `NoPenalty()`.
- `penwt`: weights for penalty term. Default is (1,1,...1,0).
- `nλ`: number of `λ` values in the sequence. Default is 100. 
- `λpath`: user-provided sequence of `λ` values in ascending order. Typically the program computes its own `λ` sequence based on `nλ`, but supplying `λpath` overrides this.
- `maxiter`: maximum number of iteration for MM loop. Default is 1000.
- `standardize`: logical flag for covariance matrix standardization. Default is `true`. If true, `V[i]` is standardized by its Frobenius norm.
- `tol`: convergence tolerance. Default is `1e-6`.

When called without any penalty, `vcselectpath!` returns the same output as `vcselect!` with `penfun=NoPenalty()` ([compare the output](#no-penalty)). 

In [41]:
vcmpath_np = deepcopy(vcm)
Σ̂path_np, Σ̂intpath_np, β̂path_np, λpath_np, objpath_np, niterspath_np = vcselectpath!(vcmpath_np)

([1.2370956021268487e-103, 1.18459422741841e-14, 1.4514980803170268e-181, 1.090535287358165e-22, 6.744718532465077e-61, 10.565155035553508, 5.131473929132298e-49, 2.875236157689849e-48, 1.941047934124898e-52, 2.5644485860721365e-14  …  6.278029617215636e-75, 1.0761872636315929e-35, 2.5959925203022874e-35, 3.260689817232068e-6, 4.19984969005698e-101, 9.337835915840925e-12, 4.030269129408247, 8.359367009877688e-118, 19.489087310922567, 5.564148840746572e-8], [1.310309666932489e-28, 3.2419000811015885e-12, 1.5702290697410488e-55, 2.672343838280577e-131, 6.211595722299537e-29, 3.159125768372407e-98, 3.722393641006552e-41, 3.8688656742473736e-100, 1.168083164394658e-25, 2.21691530955166e-62  …  1.9204496423522967e-66, 6.808362980686468e-47, 2.0580025119265423e-40, 3.4591922795688957e-62, 8.476522782648709e-144, 9.829626888036323e-70, 2.2109686410414783e-37, 4.3962325743466516e-48, 3.905521953657742e-76, 3.708715008326754], [-0.053324093482632964], [0.0], -6.465972564484275, 176)

In [42]:
Σ̂path_np

151-element Array{Float64,1}:
  1.2370956021268487e-103
  1.18459422741841e-14   
  1.4514980803170268e-181
  1.090535287358165e-22  
  6.744718532465077e-61  
 10.565155035553508      
  5.131473929132298e-49  
  2.875236157689849e-48  
  1.941047934124898e-52  
  2.5644485860721365e-14 
  5.690783579703894e-141 
  5.449013992876955e-117 
  2.5104793319889334e-123
  ⋮                      
  2.2447558973276592e-118
  1.4579400632656927e-39 
  6.278029617215636e-75  
  1.0761872636315929e-35 
  2.5959925203022874e-35 
  3.260689817232068e-6   
  4.19984969005698e-101  
  9.337835915840925e-12  
  4.030269129408247      
  8.359367009877688e-118 
 19.489087310922567      
  5.564148840746572e-8   

In [43]:
Σ̂intpath_np

150-element Array{Float64,1}:
 1.310309666932489e-28  
 3.2419000811015885e-12 
 1.5702290697410488e-55 
 2.672343838280577e-131 
 6.211595722299537e-29  
 3.159125768372407e-98  
 3.722393641006552e-41  
 3.8688656742473736e-100
 1.168083164394658e-25  
 2.21691530955166e-62   
 7.970281510431552e-107 
 8.352279869688793e-116 
 6.3034711280692e-190   
 ⋮                      
 1.2394173201953727e-16 
 4.454553258162834e-75  
 1.9204496423522967e-66 
 6.808362980686468e-47  
 2.0580025119265423e-40 
 3.4591922795688957e-62 
 8.476522782648709e-144 
 9.829626888036323e-70  
 2.2109686410414783e-37 
 4.3962325743466516e-48 
 3.905521953657742e-76  
 3.708715008326754      

Now let us call `vcselectpath!` with `penfun=L1Penalty()`. We will specify the number of regularization parameters to be 5, i.e. `nλ=5`. 

In [46]:
vcm_path = deepcopy(vcm)
Σ̂path, Σ̂intpath, β̂path, λpath, objpath, niterspath = vcselectpath!(vcm_path; nλ=5,
    penfun=L1Penalty());

Output of `vcselectpath!` includes

* path for main effect variance components: $(i,j)$-th element corresponds to $i$-th variance component (main effect) at `λpath[j]`, $i=1,\ldots, 150, \; j=1,\ldots, 5$. The last row is for the residual variance component $\sigma_0^2.$ 

In [47]:
Σ̂path

151×5 Array{Float64,2}:
  1.2371e-103   0.0        0.0        0.0        0.0      
  1.18459e-14   0.0        0.0        0.0        0.0      
  1.4515e-181   0.0        0.0        0.0        0.0      
  1.09054e-22   0.0        0.0        0.0        0.0      
  6.74472e-61   0.0        0.0        0.0        0.0      
 10.5652        0.0        0.0        0.0        0.0      
  5.13147e-49   0.0        0.0        0.0        0.0      
  2.87524e-48   0.0        0.0        0.0        0.0      
  1.94105e-52   0.0        0.0        0.0        0.0      
  2.56445e-14   0.0        0.0        0.0        0.0      
  5.69078e-141  0.0        0.0        0.0        0.0      
  5.44901e-117  0.0        0.0        0.0        0.0      
  2.51048e-123  0.0        0.0        0.0        0.0      
  ⋮                                                       
  2.24476e-118  0.0        0.0        0.0        0.0      
  1.45794e-39   0.0        0.0        0.0        0.0      
  6.27803e-75   0.0        0.0  

* path for interaction effect variance components: $(i,j)$-th element corresponds to $i$-th variance component (interaction effect) at `λpath[j]`, $i=1,\ldots, 150, \; j=1,\ldots, 5$. 

In [48]:
Σ̂intpath

150×5 Array{Float64,2}:
 1.31031e-28   0.0      0.0       0.0       0.0     
 3.2419e-12    0.0      0.0       0.0       0.0     
 1.57023e-55   0.0      0.0       0.0       0.0     
 2.67234e-131  0.0      0.0       0.0       0.0     
 6.2116e-29    0.0      0.0       0.0       0.0     
 3.15913e-98   0.0      0.0       0.0       0.0     
 3.72239e-41   0.0      0.0       0.0       0.0     
 3.86887e-100  0.0      0.0       0.0       0.0     
 1.16808e-25   0.0      0.0       0.0       0.0     
 2.21692e-62   0.0      0.0       0.0       0.0     
 7.97028e-107  0.0      0.0       0.0       0.0     
 8.35228e-116  0.0      0.0       0.0       0.0     
 6.30347e-190  0.0      0.0       0.0       0.0     
 ⋮                                                  
 1.23942e-16   0.0      0.0       0.0       0.0     
 4.45455e-75   0.0      0.0       0.0       0.0     
 1.92045e-66   0.0      0.0       0.0       0.0     
 6.80836e-47   0.0      0.0       0.0       0.0     
 2.058e-40     0.0    

* mean regression coefficient path: $j$-th element in `β̂path` corresponds to $p$-by-$d$ coefficients at `λpath[j]`.

In [49]:
β̂path

1×5 Array{Float64,2}:
 -0.0533241  -0.0606813  -0.0650561  -0.0750272  -0.159617

* grid of $\lambda$ values used 

In [50]:
 λpath

0.0:16.36382241215168:65.45528964860672

* final objective values at each $\lambda$

In [51]:
objpath

5-element Array{Float64,1}:
  -6.465972564484275
  64.39600968290858 
  80.00239479860264 
  91.66192019196805 
 100.70318340750586 

* number of iterations at each $\lambda$

In [52]:
niterspath

5-element Array{Int64,1}:
 176
 107
  25
  31
  92

### Visualize 